IMPORTS

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import ast
import re

BASES

MERGES et BASES

In [2]:
df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/343300649.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')


In [3]:
df_title_basics = df_title_basics.replace('\\N', np.nan)

In [4]:
df_title_ratings = pd.read_csv('BD/title.ratings.tsv.gz', compression = 'gzip', sep = '\t')

In [5]:
df_title_ratings = df_title_ratings.replace('\\N', np.nan)

In [6]:
df1 = pd.merge(df_title_basics,
               df_title_ratings,
               how = 'outer',
               on = 'tconst')

In [7]:
df_title_ratings, df_title_basics = 0, 0

In [8]:
df_title_akas = pd.read_csv('BD/title.akas.tsv.gz', compression = 'gzip', sep = '\t')

In [9]:
df_title_akas = df_title_akas.replace('\\N', np.nan)

In [10]:
df_3 = pd.merge(df1,
                df_title_akas,
                how = 'outer',
                left_on = 'tconst',
                right_on = 'titleId')

In [11]:
df_3 = df_3[(df_3['titleType'] == 'movie')]

In [12]:
df1, df_title_akas = 0, 0

In [13]:
df_tmbd_full = pd.read_csv('BD/tmdb_full.csv')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/1088705580.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmbd_full = pd.read_csv('BD/tmdb_full.csv')


In [14]:
df_tmbd_full = df_tmbd_full.replace('\\N', np.nan)

In [15]:
df_films = pd.merge(df_tmbd_full,
                df_3,
                how = 'outer',
                left_on = 'imdb_id',
                right_on = 'tconst')

In [16]:
df_tmbd_full, df_3 = 0, 0

COLONNES NUMERIQUES

In [17]:
# On remplit les cellules vides par 0

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].fillna(0)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/2166249025.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].fillna(0)


In [18]:
# On remplace les '\\N' par 0

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].replace('\\N', 0)

In [19]:
# On met ces colonnes au format numérique

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].apply(pd.to_numeric)

In [20]:
# On transforme les types de données pour 'release_date'

df_films['release_date'] = pd.to_datetime(df_films['release_date'])
df_films['release_date'] = df_films['release_date'].replace('\\N', 0)
df_films['release_date'] = df_films['release_date'].dt.year

In [21]:
# On transforme les types de données pour 'startYear'

df_films['startYear'] = df_films['startYear'].replace('\\N', 0)
df_films['startYear'] = pd.to_numeric(df_films['startYear'], downcast="integer")

In [22]:
# On crée la colonne year_exact

df_films['year_exact'] = df_films[['release_date', 'startYear']].max(axis = 1)

In [23]:
# On crée la colonne 'Decennie'

df_films['Decennie'] = ''
df_films['year_exact'] = df_films['year_exact'].fillna('0')
df_films['year_exact'] = df_films['year_exact'].replace('nan', '0')
df_films['year_exact'] = df_films['year_exact'].astype(str)

In [24]:
df_films['Decennie'] = df_films['year_exact'].str[:3] + "0"
# df_films['Decennie'] = df_films['Decennie'].replace("0.00's", None)
df_films['year_exact'] = pd.to_numeric(df_films['year_exact'])
df_films['Decennie'] = pd.to_numeric(df_films['Decennie'])

RUNTIME

In [25]:
# On met de côté les valeurs nulles

df_null1 = df_films[((df_films['runtimeMinutes'].isna()) & (df_films['runtime'].isna()))]
df_null2 = df_films[((df_films['runtimeMinutes'] == 0) & (df_films['runtime'] == 0))]
df_null3 = df_films[((df_films['runtimeMinutes'] == 0) & (df_films['runtime'].isna()))]
df_null4 = df_films[((df_films['runtimeMinutes'].isna()) & (df_films['runtime'] == 0))]

In [26]:
df_null = pd.concat([df_null1, df_null2, df_null3, df_null4])

In [27]:
df_null1, df_null2, df_null3, df_null4 = 0, 0, 0, 0

In [28]:
# On prend les valeurs utiles

df_utile = df_films[~((df_films['runtimeMinutes'].isna()) & (df_films['runtime'].isna()))]
df_utile = df_utile[~((df_utile['runtimeMinutes'] == 0) & (df_utile['runtime'] == 0))]
df_utile = df_utile[~((df_utile['runtimeMinutes'] == 0) & (df_utile['runtime'].isna()))]
df_utile = df_utile[~((df_utile['runtimeMinutes'].isna()) & (df_utile['runtime'] == 0))]

In [29]:
# Calcul différence de runtime des 2 tables

df_utile['difference_runtime'] = abs(df_utile['runtime'] - df_utile['runtimeMinutes'])

In [30]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_utile[(df_utile['runtimeMinutes'] == df_utile['runtime'])]

In [31]:
# Création colonne RuntimeExact
df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']

df_a_travailler_1 = df_utile[~(df_utile['runtimeMinutes'] == df_utile['runtime'])]

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/3570380868.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']


In [32]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]

In [33]:
# On conserve la donnée existante

df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/3521993277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)


In [34]:
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]
df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/1261991755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])


In [35]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_runtime'] <= 20]

In [36]:
# On prend comme base la valeur max des 2 colonnes

df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/880983650.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [37]:
df_a_travailler_3 = df_a_travailler_2[~(df_a_travailler_2['difference_runtime'] <= 20)]

In [38]:
# Pour le restant des données, on conserve la valeur max des 2 colonnes

df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)
df_a_garder_4 = df_a_travailler_3

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/264539281.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [39]:
df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])

In [40]:
df_a_travailler_1, df_a_travailler_2, df_a_travailler_3 = 0, 0, 0
df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4 = 0, 0, 0, 0

In [41]:
df_films = pd.concat([df_final, df_null])

In [42]:
df_final, df_null = 0, 0

VOTES

In [43]:
# On met de côté les valeurs inutiles

df_null5 = df_films[((df_films['vote_average'].isna()) & (df_films['averageRating'].isna()))]
df_null6 = df_films[((df_films['vote_average'] == 0) & (df_films['averageRating'] == 0))]
df_null7 = df_films[((df_films['vote_average'] == 0) & (df_films['averageRating'].isna()))]
df_null8 = df_films[((df_films['vote_average'].isna()) & (df_films['averageRating'] == 0))]

In [44]:
df_null9 = pd.concat([df_null5, df_null6, df_null7, df_null8])

In [45]:
df_null5, df_null6, df_null7, df_null8 = 0, 0, 0, 0

In [46]:
# On prend les valeurs utiles

df_ratings = df_films[~((df_films['vote_average'].isna()) & (df_films['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'] == 0))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'].isna()) & (df_ratings['averageRating'] == 0))]

In [47]:
df_ratings['vote_average'] = round(df_ratings['vote_average'], 1)

In [48]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_ratings[df_ratings['averageRating'] == df_ratings['vote_average']]
df_a_travailler_1 = df_ratings[~(df_ratings['averageRating'] == df_ratings['vote_average'])]

In [49]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[(df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0)]
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0))]

In [50]:
df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/2546417221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])


In [51]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] <= 1]
df_a_travailler_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] > 1]

In [52]:
df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
df_a_travailler_3['vote_exact'] = round(df_a_travailler_3['vote_exact'], 1)
df_a_garder_4 = df_a_travailler_3

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/4062215657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/4062215657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travaill

In [53]:
# On crée la colonne vote_exact pour garder la moyenne des votes

df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_3['vote_exact'] = df_a_garder_3[['averageRating', 'vote_average']].mean(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/3653439724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/3653439724.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/36534397

In [54]:
# On concatène tous les tableaux conservés

df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])
df_final['arrondi_vote_exact'] = round(df_final['vote_exact'], 0)

In [55]:
df_a_travailler_1, df_a_travailler_2, df_a_travailler_3 = 0, 0, 0
df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4 = 0, 0, 0, 0

In [56]:
df_films = pd.concat([df_final, df_null9])

In [57]:
df_final, df_null9 = 0, 0

GENRE

In [58]:
df_films = df_films.rename(columns = {'genres_x' : 'genres_y', 'genres_y' : 'genres_x'})

In [59]:
df_films[df_films[['genres_y', 'genres_x']] == '\\N'] = df_films[df_films[['genres_y', 'genres_x']] == '\\N'].replace('\\N', np.nan)

In [60]:
def transfo_liste(x):
  if isinstance(x, str):
    return ast.literal_eval(x)
  else:
    return x
  
df_films['genres_y'] = df_films['genres_y'].apply(transfo_liste)

In [61]:
df_films['genres_y'] = df_films['genres_y'].apply(lambda x: x if (type(x) == list) else [])

In [62]:
df_films['genres_x'] = df_films['genres_x'].str.replace(",", ", ")

In [63]:
df_films['genres_x_list'] = df_films['genres_x'].fillna('').str.split(', ')

In [64]:
df_films['genre'] = df_films.apply(lambda row: list(set(row['genres_y'] + row['genres_x_list'])),axis=1)

In [65]:
df_films['genre'] = df_films['genre'].apply(lambda x: x if x != [] else np.nan)

In [66]:
df_films = df_films.drop(['genres_x_list', 'genres_x', 'genres_y'], axis = 1)

TITLE

In [67]:
# df_films['title'] = df_films.apply(lambda row : row['title_y'] if (row['region'].str.contains('FR') == True) else row['title_y'].fillna(row['title_x']).fillna(row['primaryTitle']).fillna(row['original_title']).fillna(row['originalTitle']))

df_films['title'] = df_films['title_y'].fillna(df_films['title_x']).fillna(df_films['primaryTitle']).fillna(df_films['original_title']).fillna(df_films['originalTitle'])
df_films = df_films.drop(['original_title', 'title_y', 'title_x', 'primaryTitle', 'originalTitle'], axis = 1)

ADULT

In [68]:
df_films['adult'] = (df_films['isAdult'] == 1) | df_films['adult']
df_films = df_films.drop(['isAdult'], axis=1)

TCONST

In [69]:
df_films['film_id'] = df_films['imdb_id'].fillna(df_films['tconst']).fillna(df_films['titleId'])

In [70]:
df_films = df_films.drop(['imdb_id', 'tconst', 'titleId'], axis = 1)

VOTE COUNT

In [71]:
df_films['vote_count_mean'] = (df_films['vote_count'] + df_films['numVotes'])/2

In [72]:
df_films = df_films.drop(['runtime', 'release_date', 'vote_average', 'numVotes', 'vote_count', 'startYear', 'endYear', 'runtimeMinutes', 'averageRating', 'difference_runtime', 'difference_vote'], axis = 1)

FINAL LANGUAGE

In [73]:
colonnes_y = ['production_countries', 'spoken_languages', 'production_companies_country']

for cl in colonnes_y:
 
    df_films[cl] = df_films[cl].apply(transfo_liste)
    df_films[cl] = df_films[cl].apply(lambda x: x if (type(x) == list) else [])

In [74]:
df_films['original_language_x_list'] = df_films['original_language'].fillna('').str.split(', ')

In [75]:
df_films['language_x_list'] = df_films['language'].fillna('').str.split(', ')

In [76]:
df_films['final_language'] = df_films.apply(lambda row: list(set(row['production_countries'] + row['spoken_languages'] + row['production_companies_country'] + row['original_language_x_list'] + row['language_x_list'])),axis=1)

In [77]:
df_films['final_language'] = df_films['final_language'].apply(lambda x: x if x != [] else np.nan)

In [78]:
df_films['final_language'] = df_films['final_language'].apply(lambda x: ', '.join(x) if type(x) == list else x)

In [79]:
df_films = df_films.drop(['original_language', 'spoken_languages', 'language', 'original_language_x_list', 'language_x_list'], axis = 1)

In [80]:
df_films = df_films[(df_films['final_language'].isna() == True) | (df_films['final_language'] == '') | (df_films['final_language'].str.contains('fr') == True) | (df_films['final_language'].str.contains('FR') == True)  | (df_films['final_language'].str.contains('us') == True) | (df_films['final_language'].str.contains('US') == True)]

CLEAN FINAL DU MERGE ET GROUP BY

In [81]:
df_str = df_films.select_dtypes(exclude = 'number')

In [82]:
df_str['language_fr'] = df_str['final_language'].apply(lambda x : 1 if ('fr' in x or 'FR' in x) else 0)

In [83]:
df_str = df_str.sort_values(by = 'language_fr', ascending = False)

In [84]:
df_str_drop_dup = df_str.drop_duplicates(subset = ['film_id'], keep = 'first')

In [85]:
df_num = pd.concat([df_films['film_id'], df_films.select_dtypes(include = 'number')], axis = 1)
df_num = df_num.drop(['revenue', 'budget'], axis = 1)

In [86]:
df_revenue_budget = df_films[['film_id', 'revenue', 'budget']]
df_revenue_budget_gb = df_revenue_budget.groupby('film_id').max()

In [87]:
df_num_gb = df_num.groupby('film_id').max()

In [88]:
df_num2 = pd.merge(df_num_gb,
                   df_revenue_budget_gb,
                   how = 'outer',
                   on = 'film_id')

In [89]:
df_films = pd.merge(df_str_drop_dup,
                   df_num2,
                   on = 'film_id',
                   how = 'left')

In [90]:
df_num2, df_str_drop_dup, df_num_gb, df_revenue_budget_gb, df_revenue_budget, df_num, df_str = 0, 0, 0, 0, 0, 0, 0

In [91]:
df_films = df_films[df_films['status'] == 'Released']

In [92]:
df_films = df_films.drop(['isOriginalTitle', 'revenue', 'budget', 'ordering', 'types', 'attributes', 'video', 'id'], axis = 1)

In [93]:
df_films = df_films[~(df_films['vote_count_mean'] == 0)] # A voir si on le conserve
df_films = df_films[~(df_films['year_exact'] == 0)]
df_films = df_films[~(df_films['runtime_exact'].isna() == True)]
df_films['region'] = df_films['region'].fillna('Unknown')

In [94]:
df_films = df_films.rename(columns = {
    'backdrop_path' : 'backdrop_path_out_KNN',
    'homepage' : 'homepage_out_KNN',
    'overview' : 'overview_out_KNN',
    'poster_path' : 'poster_path_out_KNN',
    'film_id' : 'film_id_out_KNN',
    'title' : 'title_out_KNN',
})

CHANGEMENT GENRE

In [95]:
tous_les_genres = set()
for genres in df_films['genre']:
  tous_les_genres.update(genres)

In [96]:
tous_les_genres.remove("")

In [97]:
for genre in tous_les_genres:
  df_films[f'genre_{genre}'] = df_films['genre'].apply(lambda x: genre in x)

In [98]:
df_films = df_films.rename({'genre' : 'genre_out_KNN'}, axis = 1)

CHANGEMENT production_countries

In [99]:
tous_les_genres = set()
for genres in df_films['production_countries']:
  tous_les_genres.update(genres)

In [100]:
for genre in tous_les_genres:
  df_films[f'production_countries_{genre}'] = df_films['production_countries'].apply(lambda x: genre in x)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/308899010.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'production_countries_{genre}'] = df_films['production_countries'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/308899010.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'production_countries_{genre}'] = df_films['production_countries'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/30889901

In [101]:
df_films = df_films.drop(['production_countries'], axis = 1)

CHANGEMENT production_companies_country

In [102]:
df_films['production_companies_country2'] = df_films['production_companies_country'].apply(lambda x : ",".join(x).lower())

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/3094971156.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films['production_companies_country2'] = df_films['production_companies_country'].apply(lambda x : ",".join(x).lower())


In [103]:
df_films['production_companies_country3'] = df_films['production_companies_country2'].apply(lambda x : x.split(','))

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/1064447305.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films['production_companies_country3'] = df_films['production_companies_country2'].apply(lambda x : x.split(','))


In [104]:
tous_les_genres = set()
for genres in df_films['production_companies_country3']:
  tous_les_genres.update(genres)

In [105]:
tous_les_genres.remove('')

In [106]:
for genre in tous_les_genres:
  df_films[f'production_companies_country_{genre}'] = df_films['production_companies_country'].apply(lambda x: genre in x)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/2202528436.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'production_companies_country_{genre}'] = df_films['production_companies_country'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/2202528436.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'production_companies_country_{genre}'] = df_films['production_companies_country'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjck

In [107]:
df_films = df_films.drop(['production_companies_country', 'production_companies_country3', 'production_companies_country2'], axis = 1)

In [108]:
df_films = df_films.rename(columns = {'final_language' : 'final_language_out_KNN', 'tagline' : 'tagline_out_KNN', 'production_companies_country FR': 'production_companies_countryfr'})

CHANGEMENT POUR LANGUE FR

In [109]:
df_films['prod_FR'] = df_films.apply(lambda row : 1 if row['production_companies_country_fr'] == True else 0, axis = 1)

CHANGEMENT POUR FILM US

In [110]:
df_films['prod_US'] = df_films.apply(lambda row : 1 if row['production_companies_country_us'] == True else 0, axis = 1)

In [111]:
df_films = df_films.drop('production_companies_country_us', axis = 1)

In [112]:
df1, df_3, df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4, df_a_travailler_1, df_a_travailler_2, df_a_travailler_3, df_final, df_null = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [113]:
df_films[df_films.select_dtypes('number') == True] = df_films[df_films.select_dtypes('number') == True].fillna(0)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2553/3129310626.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_films[df_films.select_dtypes('number') == True] = df_films[df_films.select_dtypes('number') == True].fillna(0)


In [114]:
df_films[['vote_exact', 'arrondi_vote_exact']] = df_films[['vote_exact', 'arrondi_vote_exact']].astype(float)

In [115]:
df_films['titleType'] = df_films['titleType'].fillna('movie')

In [116]:
df_num = df_films.select_dtypes(include = 'number')
df_num = df_num.fillna(0)
df_str = df_films.select_dtypes(exclude = 'number')
df_str = df_str.fillna("Unknown")

In [117]:
df_films = pd.concat([df_num, df_str], axis = 1)

In [118]:
df_str, df_num = 0, 0

In [119]:
df_films = df_films[(df_films['final_language_out_KNN'].str.contains('fr') == True) | (df_films['final_language_out_KNN'].str.contains('FR') == True)]

In [120]:
def len_pandas(x):
    
    x_escaped = re.escape(x)
    return df_films[df_films['production_companies_name2'].fillna('').str.contains(x_escaped, na=False, regex=True)].shape[0]

In [122]:
df_films = df_films.rename(columns = {'production_companies_name': 'production_companies_name_out_KNN',
                          'region' : 'region_out_KNN'})

In [123]:
df_films['production_companies_name2'] = df_films['production_companies_name_out_KNN'].apply(lambda x: x.replace('"', "").replace("[", '').replace("]", '').replace("'", '').replace(" ", '').lower())
df_films['production_companies_name3'] = df_films['production_companies_name2'].apply(lambda x : x.split(','))

In [124]:
tous_les_genres = set()

In [125]:
for element in df_films['production_companies_name3']:
  tous_les_genres.update(element)

tous_les_genres_list = list(tous_les_genres)

In [126]:
tous_les_genres_list.remove("")
tous_les_genres_list.sort()
tous_les_genres_list2 = []

In [127]:
for element in tous_les_genres_list:
    if len_pandas(element) >= 500:
       tous_les_genres_list2.append(element)

In [128]:
last_list = []
for element in tous_les_genres_list2:
    if len(element) > 3:
        last_list.append(element)

In [129]:
for element in last_list:
  df_films[f'production_companies_name_{element}'] = df_films['production_companies_name2'].apply(lambda x: element in x)

In [130]:
df_films['production_companies_name_Marvel'] = df_films['production_companies_name2'].apply(lambda x: 'Marvel' in x)

In [131]:
df_films = df_films.drop(['production_companies_name2', 'production_companies_name3'], axis = 1)

In [132]:
df_films_not_bool = df_films.select_dtypes(exclude = 'bool')

In [133]:
df_films_bool = df_films.select_dtypes(include = 'bool')

In [134]:
df_films_bool = df_films_bool.astype(str)

In [135]:
df_films_bool = df_films_bool.replace('True', '1').replace('False', '0')

In [136]:
df_films_bool = df_films_bool.astype(int)

In [137]:
df_films = pd.concat([df_films_not_bool, df_films_bool], axis = 1)

In [138]:
df_films_not_bool, df_films_bool = 0, 0

EXPORT

In [139]:
df_films.to_csv('P2_G5_films.csv.gz', index=False, compression='gzip')